In [50]:
import string
import matplotlib.pyplot as plt
%matplotlib inline
import nltk
from nltk.corpus import stopwords
import scipy

import sys, os, re, csv, codecs, numpy as np, pandas as pd
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model
from keras import initializers, regularizers, constraints, optimizers, layers

In [51]:
class trainmodel:
                    
    def __init__(self):        
        self.train_df = self.test_df = self.Y = pd.DataFrame()
        self.train_df = pd.read_csv("../../data/processed/train.csv")
        self.test_df = pd.read_csv("../../data/processed/test.csv")         
        
    def transform_target(self):
        
        le  = LabelBinarizer()
        cat_arr = le.fit_transform(self.train_df['cat'])
        cat_df = pd.DataFrame(data=cat_arr,columns=le.classes_)         
        self.train_df['subcat'] = np.where(pd.isnull(self.train_df['subcat']),'NaN',self.train_df['subcat'])
        subcat_arr = le.fit_transform(self.train_df['subcat'])
        subcat_df = pd.DataFrame(data=subcat_arr,columns=le.classes_)         
        self.Y = pd.concat([cat_df,subcat_df],axis=1)        
        self.Y.drop('NaN',inplace=True,axis=1)
        #print('Y value', self.Y.columns)
        
    def prepare_data(self):
        
        col_todrop = ['cat','subcat','Unnamed: 0','document_id']
        for col in col_todrop:
            self.train_df.drop(col,axis=1,inplace=True)   
            
            
    def createModelKeras(self):
        
        max_features = 20000
        tokenizer = Tokenizer(num_words=max_features)
        tokenizer.fit_on_texts(list(self.train_df['content']))
        list_tokenized_train = tokenizer.texts_to_sequences(self.train_df['content'])
        list_tokenized_test = tokenizer.texts_to_sequences(self.test_df['content'])
        
        maxlen = 200
        X_t = pad_sequences(list_tokenized_train, maxlen=maxlen)
        X_te = pad_sequences(list_tokenized_test, maxlen=maxlen)
        
        X_train, X_test, Y_train, Y_test = train_test_split(X_t, self.Y, 
                                                            test_size=0.33, random_state=42)  
        
        inp = Input(shape=(maxlen, ))
        embed_size = 128
        x = Embedding(max_features, embed_size)(inp)
        x = LSTM(60, return_sequences=True,name='lstm_layer')(x)
        x = GlobalMaxPool1D()(x)        
        x = Dropout(0.1)(x)
        x = Dense(50, activation="relu")(x)
        x = Dropout(0.1)(x)
        x = Dense(18, activation="sigmoid")(x)
        model = Model(inputs=inp, outputs=x)
        model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
        batch_size = 32
        epochs = 2
        model.fit(X_t,self.Y, batch_size=batch_size, epochs=epochs, validation_split=0.1)
        predicition_test = model.predict(X_test)
        print('validation dataset accuracy is {}'.format(accuracy_score(Y_test, predicition_test)))           
        print('Hamming loss is {}'.format(hamming_loss(Y_test, predicition_test)))
        print('F1 Score is {}'.format(f1_score(Y_test, predicition_test,average='samples')))
        model.summary()
        
        
    def prepare_validationset(self):        
        
        X_train, X_test, Y_train, Y_test = train_test_split(X_t, self.Y, 
                                                            test_size=0.33, random_state=42)           
        
        return(X_train, X_test, Y_train, Y_test)

    def createmodel(self):
        
        X_train, X_test, Y_train, Y_test =  self.prepare_validationset()
        #X_train, X_test, Y_train, Y_test = train_test_split(self.train_df['content'], self.Y,test_size=0.33, random_state=42)                             
        X_train_transposed =X_train.T
        self.pipe = Pipeline([
                    ('cv', CountVectorizer(ngram_range=(1, 3))),
                    ('tfidf', TfidfTransformer()),
                    ('LP_GNB', LabelPowerset(GaussianNB()))])
        # train
        self.pipe.fit(X_train_transposed, Y_train)
        
        # predict
        prediction = self.pipe.predict(X_test)
         
        print('validation dataset accuracy is {}'.format(accuracy_score(Y_test, prediction)))           
        print('Hamming loss is {}'.format(hamming_loss(Y_test, prediction)))
        print('F1 Score is {}'.format(f1_score(Y_test, prediction,average='samples')))
        
        return()
    
    def termfreq(self):
        
        tfidfvec = TfidfVectorizer(ngram_range=(1,3), strip_accents='unicode',
                       lowercase =True, analyzer='word', token_pattern=r'\w+',
                       stop_words = 'english')
        
        tfidf = tfidfvec.fit_transform(self.train_df['content'])  
        self.train_df.drop('content',inplace=True,axis=1)
        df1 = pd.DataFrame(tfidf.toarray(), columns=tfidfvec.get_feature_names())
        self.train_df = pd.concat([self.train_df, df1], axis=1)
        svdT = TruncatedSVD(n_components=4000)
        self.train_df = svdT.fit_transform(self.train_df)        
        
        test_tfidf = tfidfvec.transform(self.test_df['content'])
        self.test_df.drop('content',inplace=True,axis=1)
        df1 = pd.DataFrame(test_tfidf.toarray(), columns=tfidfvec.get_feature_names())
        self.test_df = pd.concat([self.test_df, df1], axis=1)
        self.test_df = svdT.transform(self.test_tfidf)
        
         
    
    def createmodel_2(self):
                        
        self.termfreq()              
        
        clf = LabelPowerset(GaussianNB())
        X_train, X_test, Y_train, Y_test =  self.prepare_validationset()
        clf.fit(X_train,Y_train)
        prediction = clf.predict(X_test)
        
        # predict
        # prediction = self.pipe.predict(X_test)
         
        print('validation dataset accuracy is {}'.format(accuracy_score(Y_test, prediction)))           
        print('Hamming loss is {}'.format(hamming_loss(Y_test, prediction)))
        print('F1 Score is {}'.format(f1_score(Y_test, prediction,average='samples')))
        
        return()
    
    def prepare_testdata(self,fname):
        
        self.test_df = pd.read_csv(fname)         
        self.test_id = self.test_df['document_id']
        
        col_todrop = ['document_id','Unnamed: 0']
        for col in col_todrop:
            self.test_df.drop(col,axis=1,inplace=True)         
    
    def predict_testdataset(self):
         
        test_predicitions = self.pipe.predict(self.test_df['content'])         
        return(test_predicitions)
    
    
    def get_testdata(self):
        return(self.test_df)
    
    def get_target(self):
        return(self.Y)
    

In [52]:
train_df = pd.read_csv("../../data/processed/train.csv")

In [53]:
model = trainmodel()
model.transform_target()

### Baseline code
validation dataset accuracy is 0.802211302211
Hamming loss is 0.0293475293475
F1 Score is 0.853808353808

### after adding the last page of the record
validation dataset accuracy is 0.815295815296
Hamming loss is 0.0282186948854
F1 Score is 0.857864357864

In [54]:
model.createModelKeras()

Train on 1890 samples, validate on 210 samples
Epoch 1/2
1890/1890 [==============================] - 6s 3ms/step - loss: 0.4382 - acc: 0.8497 - val_loss: 0.2779 - val_acc: 0.9021

ValueError: Classification metrics can't handle a mix of multilabel-indicator and continuous-multioutput targets

In [ ]:
model.prepare_data()
model.createmodel() 

In [ ]:
#Read the data for which the label to be predicted......
model.prepare_testdata("../../data/processed/test.csv")

#test_result is the predicted label 
test_result = model.predict_testdataset()

In [ ]:
#Convert the binary n-array to the corresponding label
#resultant array will have two columns - cat, subcat

data = np.zeros(shape=[test_result.shape[0],2])
data_str = data.astype(str)

cx = scipy.sparse.coo_matrix(test_result)
for i,j,v in zip(cx.row, cx.col, cx.data):      
    if(data_str[i][0] == '0.0'):
        #print("Inside if stmt ...... ",i,j)
        data_str[i][0] = commonfunction.getlabelname(j)
    else:
        #print("Inside if else ...... ",i,j)
        data_str[i][1] = commonfunction.getlabelname(j)
     
    

In [ ]:
test_df = pd.read_csv("../../data/processed/test.csv")
test_id = test_df['document_id']
label_df = pd.DataFrame({' ':test_id,'cat':data_str[:,0],'subcat':data_str[:,1]})
label_df.to_csv("../../data/processed/predicted.csv")